In [52]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
%load_ext tensorboard

import datetime


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [53]:
print(tf.__version__)

2.1.0


In [54]:
dataset = pd.read_csv('Customer_Churn_Modelling.csv')
dataset.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [55]:
X = dataset.drop(labels=['CustomerId', 'Surname', 'RowNumber', 'Exited'], axis = 1)
y = dataset['Exited']
X.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [56]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [57]:
label1 = LabelEncoder()
X['Geography'] = label1.fit_transform(X['Geography'])
label = LabelEncoder()
X['Gender'] = label.fit_transform(X['Gender'])
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10


In [58]:
X = pd.get_dummies(X, drop_first=True, columns=['Geography'])
X.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_1,Geography_2
0,619,0,42,2,0.00,1,1,1,101348.88,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,1


#### Feature Standardization

In [59]:
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [60]:
X_train

array([[-1.24021723, -1.09665089,  0.77986083, ...,  1.64099027,
        -0.57812007, -0.57504086],
       [ 0.75974873,  0.91186722, -0.27382717, ..., -1.55587522,
         1.72974448, -0.57504086],
       [-1.72725557, -1.09665089, -0.9443559 , ...,  1.1038111 ,
        -0.57812007, -0.57504086],
       ...,
       [-0.51484098,  0.91186722,  0.87565065, ..., -1.01507508,
         1.72974448, -0.57504086],
       [ 0.73902369, -1.09665089, -0.36961699, ..., -1.47887193,
        -0.57812007, -0.57504086],
       [ 0.95663657,  0.91186722, -1.32751517, ...,  0.50945854,
        -0.57812007,  1.73900686]])

### Build Artificial Nural Network (ANN)


In [76]:
# Clear any logs from previous runs
!rm -rf ./logs/ 
model = Sequential()
model.add(Dense(X.shape[1], activation='relu', input_dim = X.shape[1]))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.fit(X_train, y_train.to_numpy(), batch_size = 10, epochs = 50, verbose = 1,
          validation_data=(X_test, y_test.to_numpy()), 
          callbacks=[tensorboard_callback])


Train on 8000 samples, validate on 2000 samples
Epoch 1/50
8000/8000 [==============================] - 1s 114us/sample - loss: 0.4477 - accuracy: 0.8070 - val_loss: 0.4189 - val_accuracy: 0.8255
Epoch 2/50
8000/8000 [==============================] - 1s 103us/sample - loss: 0.3965 - accuracy: 0.8369 - val_loss: 0.3762 - val_accuracy: 0.8475
Epoch 3/50
8000/8000 [==============================] - 1s 133us/sample - loss: 0.3621 - accuracy: 0.8546 - val_loss: 0.3560 - val_accuracy: 0.8550
Epoch 4/50
8000/8000 [==============================] - 1s 108us/sample - loss: 0.3492 - accuracy: 0.8577 - val_loss: 0.3522 - val_accuracy: 0.8540
Epoch 5/50
8000/8000 [==============================] - 1s 129us/sample - loss: 0.3432 - accuracy: 0.8606 - val_loss: 0.3494 - val_accuracy: 0.8590
Epoch 6/50
8000/8000 [==============================] - 1s 109us/sample - loss: 0.3376 - accuracy: 0.8593 - val_loss: 0.3460 - val_accuracy: 0.8610
Epoch 7/50
8000/8000 [==============================] - 1s 139us

In [69]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 4787), started 0:13:53 ago. (Use '!kill 4787' to kill it.)

In [77]:
y_pred = model.predict_classes(X_test)
y_pred


array([[0],
       [0],
       [0],
       ...,
       [0],
       [1],
       [0]], dtype=int32)

In [78]:
model.evaluate(X_test, y_test.to_numpy())


2000/2000 [==============================] - 0s 39us/sample - loss: 0.3606 - accuracy: 0.8480


[0.36055211269855497, 0.848]

In [79]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [80]:
confusion_matrix(y_test, y_pred)

array([[1499,   94],
       [ 210,  197]])

In [81]:
accuracy_score(y_test, y_pred)*100


84.8